In [23]:
import json
import time

class Product:
    def __init__(self, product_id, name, price, quantity):
        self.product_id = product_id
        self.name = name
        self.price = price
        self.quantity = quantity

    def update_quantity(self, sold_quantity):
        if sold_quantity <= self.quantity:
            self.quantity -= sold_quantity
            return True
        return False

    def to_dict(self):
        """Convert product details to a dictionary for JSON serialization."""
        return {
            "Name": self.name,
            "Price": self.price,
            "Qn": self.quantity
        }

class Inventory:
    def __init__(self, json_file):
        self.json_file = json_file
        self.products = self.load_inventory()

    def load_inventory(self):
        """Load inventory from JSON file."""
        with open(self.json_file, "r") as file:
            data = json.load(file)
        return {
            key: Product(
                product_id=key,
                name=value["Name"],
                price=value["Price"],
                quantity=value["Qn"]
            )
            for key, value in data.items()
        }

    def update_inventory(self):
        """Write the updated inventory back to the JSON file."""
        with open(self.json_file, "w") as file:
            json.dump(
                {key: product.to_dict() for key, product in self.products.items()},
                file,
                indent=4
            )

class Customer:
    def __init__(self, name, email, phone, sex):
        self.name = name
        self.email = email
        self.phone = phone
        self.sex = sex.upper()

class Sale:
    def __init__(self, customer, product, quantity):
        self.customer = customer
        self.product = product
        self.quantity = quantity
        self.time = time.ctime()

    def calculate_total(self):
        base_price = self.product.price * self.quantity
        discount = base_price * 0.1 if base_price > 500 else 0
        tax = base_price * 0.15
        return base_price - discount + tax

    def generate_receipt(self):
        total = self.calculate_total()
        if total > 500:
            receipt = (
                f"Customer      : {self.customer.name}\n"
                f"Product       : {self.product.name}\n"
                f"Quantity      : {self.quantity}\n"
                f"Date          : {self.time}\n"
                f"{'-'*50}\n"
                f"Discount(10%) : {self.product.price * self.quantity * 0.1}\n"
                f"SGST(7.5%)    : {self.product.price * self.quantity * 0.075}\n"
                f"CGST(7.5%)    : {self.product.price * self.quantity * 0.075}\n"
                f"{'-'*50}\n"
                f"Total Bill    : ₹{total:.2f}\n"
                
            )
        else:
             receipt = (
                f"Customer      : {self.customer.name}\n"
                f"Product       : {self.product.name}\n"
                f"Quantity      : {self.quantity}\n"
                f"Date          : {self.time}\n"
                f"{'-'*50}\n"
                f"Discount(10%) : {self.product.price * self.quantity * 0}\n"
                f"SGST(7.5%)    : {self.product.price * self.quantity * 0.075}\n"
                f"CGST(7.5%)    : {self.product.price * self.quantity * 0.075}\n"
                f"{'-'*50}\n"
                f"Total Bill    : ₹{total:.2f}\n"
            )
        return receipt

    def log_sale(self, sale_file):
        """Append the sale details to a CSV file."""
        total = self.calculate_total()
        sale_entry = (
            f"{self.customer.name},{self.customer.email},{self.customer.phone},{self.customer.sex},"
            f"{self.product.name},{self.quantity},{total:.2f},{self.time}\n"
        )
        with open(sale_file, "a") as file:
            file.write(sale_entry)



inventory_file = "Record.json"
sales_file = "Sales2.csv"


inventory = Inventory(inventory_file)


print("-" * 22 + "MENU" + "-" * 24)
for key, product in inventory.products.items():
    print(f"{key} : {product.name} | ₹{product.price} | Quantity: {product.quantity}")
print("\n")


customer_name = input("Enter your name: ")
customer_email = input("Enter your email: ")
customer_phone = input("Enter your phone number: ")
customer_sex = input("Enter your sex (M/F/O): ")
customer = Customer(customer_name, customer_email, customer_phone, customer_sex)


product_id = input("Enter the Product ID: ")
quantity = int(input("Enter the quantity: "))

if product_id in inventory.products:
    product = inventory.products[product_id]

    if product.update_quantity(quantity):
        sale = Sale(customer, product, quantity)
        print("-"*50)
        print("BILL".center(50,"-"))
        print(sale.generate_receipt())
        sale.log_sale(sales_file)
        inventory.update_inventory()
    else:
        print("Insufficient stock!")
else:
    print("Invalid Product ID!")


----------------------MENU------------------------
1001 : 5 Star | ₹15 | Quantity: 198
1002 : Eclairs | ₹2 | Quantity: 1982
1003 : Galaxy | ₹20 | Quantity: 200
1004 : Crispelo | ₹30 | Quantity: 277
1005 : Chocolate Cake | ₹300 | Quantity: 58




Enter your name:  Ankit Kumar
Enter your email:  ankit@jkl.com
Enter your phone number:  183682982
Enter your sex (M/F/O):  M
Enter the Product ID:  1003
Enter the quantity:  5


--------------------------------------------------
-----------------------BILL-----------------------
Customer      : Ankit Kumar
Product       : Galaxy
Quantity      : 5
Date          : Wed Nov 27 16:19:01 2024
--------------------------------------------------
Discount(10%) : 0
SGST(7.5%)    : 7.5
CGST(7.5%)    : 7.5
--------------------------------------------------
Total Bill    : ₹115.00

